In [1]:
import os
from time import time
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import GSHHSFeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import xarray as xr
from scipy import interpolate
from datetime import date
import math
from scipy import signal

C:\Users\oa18724\Anaconda3\envs\dl_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [191]:
omega = 7.2921e-5

In [2]:
def plot_drifter_trajectories(df, direction=None, global_extent=False):
    ids = df['id'].unique()
    for id in ids:
        # df_id = df_id[1:-1]
        df_id = df.loc[df['id']==id]
        fig = plt.figure(figsize=(20,12))
        crs = ccrs.PlateCarree()
        ax = fig.add_subplot(1, 1, 1, projection=crs)
        ax.add_feature(GSHHSFeature(scale='intermediate', facecolor='lightgrey', linewidth=0.2))
        ax.set_xticks(np.linspace(-180, 180, 19), crs=crs)
        ax.set_yticks(np.linspace(-90, 90, 19), crs=crs)
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)
        if global_extent:
            ax.set_global()
        # ax.gridlines()
        if direction=='v':
            plt.scatter(x=df_id.lon, y=df_id.lat, s=5, c=df_id.vn, transform=crs, cmap='jet')
        elif direction=='u':
            plt.scatter(x=df_id.lon, y=df_id.lat, s=5, c=df_id.ve, transform=crs, cmap='jet')
        elif direction=='corr_filt_v':
            plt.scatter(x=df_id.filt_lon, y=df_id.filt_lat, s=5, c=df_id.corr_vn, transform=crs, cmap='jet')
        elif direction=='corr_filt_u':
            plt.scatter(x=df_id.filt_lon, y=df_id.filt_lat, s=5, c=df_id.corr_ve, transform=crs, cmap='jet')
        else:
            plt.scatter(x=df_id.lon, y=df_id.lat, s=5, c=df_id.spd, transform=crs, cmap='jet')
            print('Displayed speed: input v for northward velocity or u for eastward')
        plt.show()

In [3]:
def plot_time_step(time_step_list, direction=None):
    fig = plt.figure(figsize=(20,12))
    crs = ccrs.PlateCarree()
    ax = fig.add_subplot(1, 1, 1, projection=crs)
    ax.add_feature(GSHHSFeature(scale='intermediate', facecolor='lightgrey', linewidth=0.2))
    ax.set_xticks(np.linspace(-180, 180, 19), crs=crs)
    ax.set_yticks(np.linspace(-90, 90, 19), crs=crs)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.set_global()
    if direction=='v':
        plt.scatter(x=time_step_list.lon, y=time_step_list.lat, s=5, c=time_step_list.vn, transform=crs, cmap='jet')
    elif direction=='u':
        plt.scatter(x=time_step_list.lon, y=time_step_list.lat, s=5, c=time_step_list.ve, transform=crs, cmap='jet')
    else:
        plt.scatter(x=time_step_list.lon, y=time_step_list.lat, s=5, c=time_step_list.spd, transform=crs, cmap='jet')
        print('Displayed speed: input v for northward velocity or u for eastward')
    plt.show()

In [4]:
def get_time_step(df, years, months, days):
    df = df.copy()
    df_list = []
    # could replace nested for loop with hrs
    for year in years:
        for month in months:
            for day in days:
                # df = df.loc[df["spd"]<999.999]
                df_t = df.loc[(df['yy']==year) & (df['mm']==month) & (df['dd']==day)]
                df_list.append(df_t)
    return df_list

In [5]:
def get_time_step_over_month(df, years, months):
    df = df.copy()
    df_list = []
    for year in years:
        for month in months:
            # df = df.loc[df["spd"]<999.999]
            df_t = df.loc[(df['yy']==year) & (df['mm']==month)]
            df_list.append(df_t)
    return df_list

In [6]:
def get_drifter_vel(df, direction=None):
    df = df.copy()
    # df = df.loc[df["spd"]<999.999]
    if direction == 'v':
        return df['vn']
    elif direction == 'u':
        return df['ve']
    else:
        return df['spd']


In [7]:
def add_hours_column(df):
    df = df.copy()
    days, mnths, yrs = df['dd'], df['mm'], df['yy']
    conv_times = []
    for yr, mnth, day in zip(yrs, mnths, days):
        t = convert_time_to_hrs(year=yr, month=mnth, day=day)
        conv_times.append(t)
    df['hrs'] = conv_times
    return df

In [8]:
def convert_time_to_hrs(year, month, day):
    d_split = math.modf(day)
    d0 = date(1900, 1, 1)
    d1 = date(year, month, int(d_split[1]))
    diff = (d1 - d0)
    hours = (diff.days + d_split[0]) * 24
    return hours

In [9]:
def read_dat_file(path, filename):
    filepath = os.path.join(os.path.normpath(path), filename)
    fid = open(filepath, mode='r')
    df = pd.read_csv(fid, sep="\s+", header=None)
    return df

In [10]:
def read_drifters_to_df(path, filename, headers):
    df = read_dat_file(path, filename)
    df.columns = headers
    df.astype(float)
    df = df.loc[df["spd"]<999.999]
    return df

In [116]:
def correct_velocities_wind_slip(df):
    df = df.copy()
    df['corr_ve'] = 0
    df['corr_vn'] = 0
    df['wind_u10'] = 0
    df['wind_v10'] = 0
    yrs = sorted(df['yy'].unique()) 
#     yrs = [1988, 1989, 1990, 1991, 1992] #----------------- remove ------------------
    print(yrs)
    for yr in yrs:
        print(yr)
        f_path = f'D:/era5/download_{yr}.nc'
        if os.path.exists(f_path):
            era5_name = f'D:/era5/download_{yr}.nc'
            wind_dataset = nc.Dataset(era5_name)
            df = subtract_windslip(df, wind_dataset)
        else:
            print(f'{yr} era5 file does not exist')
    return df

In [117]:
def subtract_windslip(df, wind_dataset):
    df = df.copy()
    times = wind_dataset.variables['time'][:]
    u10, v10 = wind_dataset.variables['u10'], wind_dataset.variables['v10']
    lats, lons = wind_dataset.variables['latitude'][:], wind_dataset.variables['longitude'][:]
    for i, time in enumerate(times):
        f_u10 = interpolate.interp2d(lons, lats, u10[i], kind='cubic')
        f_v10 = interpolate.interp2d(lons, lats, v10[i], kind='cubic')
        df_t = df.loc[df["hrs"]==time]
        indices = df.index[df["hrs"]==time]
        # any_true = np.any(df["hrs"]==time)
        # if any_true:
            # print(i, f"there was a match!!!")
        u10_new = df_t.apply(lambda x: f_u10(x['lon'], x['lat'])[0], axis=1)
        v10_new = df_t.apply(lambda x: f_v10(x['lon'], x['lat'])[0], axis=1)
        wind_slip_u = u10_new * 0.07
        wind_slip_v = v10_new * 0.07
        df.loc[indices,'corr_ve'] = df_t['ve'] - wind_slip_u
        df.loc[indices,'corr_vn'] = df_t['vn'] - wind_slip_v
        df.loc[indices, 'wind_u10'] = u10_new
        df.loc[indices, 'wind_v10'] = v10_new
    return df

In [13]:
def filter_inertial_motion(df):
    # print('-- filtering inertial motion --')
    df = df.copy()
    ids = df['id'].unique()

    for id in ids:
        df_id = df.loc[df['id']==id]
        indices = df.index[df["id"]==id]

        corr_vn = np.array(df_id['corr_vn'])
        corr_ve = np.array(df_id['corr_ve'])
        lons = np.array(df_id['lon'])
        lats = np.array(df_id['lat'])
        lats[lats == 0] = 0.01
        n = len(lats)
        f = 2*omega*np.sin(lats)
        T = (2*np.pi/(f))/3600
        w = (np.floor(np.abs(T/6))).astype(int)

        filt_lats = np.zeros(n)
        filt_lons = np.zeros(n)
        filt_ve = np.zeros(n)
        filt_vn = np.zeros(n)
        for i in range(n):
            left_w = min(w[i], i)
            right_w = min(w[i], n-i-1)
            filt_lats[i] = np.mean(lats[i-left_w:i+right_w])
            if not filt_lats[i]:
                print(f'lat[{i}] is empty') 
            filt_lons[i] = np.mean(lons[i-left_w:i+right_w])
            filt_ve[i] = np.mean(corr_ve[i-left_w:i+right_w])
            filt_vn[i] = np.mean(corr_vn[i-left_w:i+right_w])
        df.loc[indices, 'filt_lat'] = filt_lats
        df.loc[indices, 'filt_lon'] = filt_lons
        df.loc[indices, 'filt_ve'] = filt_ve
        df.loc[indices, 'filt_vn'] = filt_vn
    return df

In [122]:
fpath = 'drifters/'
fnames = ['buoydata_1_5000.dat', 'buoydata_5001_10000.dat', 'buoydata_10001_15000.dat','buoydata_15001_dec21.dat']    
    
headers = ['id', 'mm', 'dd', 'yy', 'lat', 'lon', 'temp', 've', 'vn', 'spd', 'var_lat', 'var_lon', 'var_temp']
df_d = read_drifters_to_df(fpath, fnames[0], headers)
df_d = df_d.drop(df_d.columns[[6,10,11,12]], axis=1)
df_d = add_hours_column(df_d)
# df_d = df_d.head(10)

In [123]:
df_d = correct_velocities_wind_slip(df_d)

[1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006]
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006


In [124]:
print(df_d)

              id  mm    dd    yy     lat      lon      ve      vn     spd  \
1        7702986   3  8.25  1988  -1.242  274.772 -49.214  36.778  61.438   
2        7702986   3  8.50  1988  -1.176  274.657 -57.514  28.439  64.161   
3        7702986   3  8.75  1988  -1.131  274.548 -48.625  28.747  56.487   
4        7702986   3  9.00  1988  -1.064  274.468 -50.787  34.399  61.340   
5        7702986   3  9.25  1988  -0.997  274.351 -59.573  34.328  68.756   
...          ...  ..   ...   ...     ...      ...     ...     ...     ...   
8334588  9815710   3  8.25  1999  12.842   61.420  -6.746  12.206  13.946   
8334589  9815710   3  8.50  1999  12.872   61.388  -3.180  12.852  13.239   
8334590  9815710   3  8.75  1999  12.892   61.408   5.135  10.721  11.887   
8334591  9815710   3  9.00  1999  12.914   61.408   3.649   8.132   8.914   
8334592  9815710   3  9.25  1999  12.924   61.422   0.824   3.329   3.429   

              hrs    corr_ve    corr_vn  wind_u10  wind_v10  
1        7729

In [125]:
df_copy = df_d.copy()

In [127]:
df_filt = filter_inertial_motion(df_d)

C:\Users\oa18724\Anaconda3\envs\dl_env\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\oa18724\Anaconda3\envs\dl_env\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [150]:
nan_ind = np.where(df_filt['filt_lat'].isna())[0]
print(print(nan_ind))
# print(df_filt.loc[557662, :])

[ 557662 2645564 3405442 3992679]
None


In [45]:
# ammend according to above
# df_filt2 = df_filt.drop([df_filt.index[259209], df_filt.index[2071742], df_filt.index[2471352]])

In [ ]:
# def compute_geostrophic_anomalies():
    

In [180]:
def compute_mean_vel(df, box_size=5):
    df = df.copy()
    df[df.columns[0]].count()
    df['vn_anom'] = 0
    df['ve_anom'] = 0
    lon_range = np.arange(0, 360, box_size)
    lat_range = np.arange(-90, 90, box_size)
#     lat_range = np.arange(np.nanmin(df['filt_lat'].to_numpy()), np.nanmax(df['filt_lat'].to_numpy()), box_size)
#     mean_vn = np.zeros((len(lon_range),len(lat_range)))
#     mean_ve = np.zeros((len(lon_range),len(lat_range)))
    print('lon_range:', lon_range, 'lat_range:', lat_range)
    
    for lon in lon_range:
        for lat in lat_range:
#             box = df.loc[(df['filt_lon']>=lon) & (df['filt_lon']<lon+5) & (df['filt_lat']>=lat) & (df['filt_lat']<lat+5)]
            indices = (df['filt_lon']>=lon) & (df['filt_lon']<lon+5) & (df['filt_lat']>=lat) & (df['filt_lat']<lat+5)
            box = df.loc[indices]
            if not box.empty:
                mean_vn = np.mean(box['filt_vn'].to_numpy())
                mean_ve = np.mean(box['filt_ve'].to_numpy())
                df.loc[indices, 'vn_anom'] = box - mean_vn
                df.loc[indices, 've_anom'] = box - mean_ve
#                 print(f'{lon}, {lat}: BOX NOT EMPTY**')
#             else:
#                 print(f'{lon}, {lat}: Box is empty.')
    return df

In [181]:
df_filt_anom = compute_mean_vel(df_filt)

lon_range: [  0   5  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85
  90  95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175
 180 185 190 195 200 205 210 215 220 225 230 235 240 245 250 255 260 265
 270 275 280 285 290 295 300 305 310 315 320 325 330 335 340 345 350 355] lat_range: [-90 -85 -80 -75 -70 -65 -60 -55 -50 -45 -40 -35 -30 -25 -20 -15 -10  -5
   0   5  10  15  20  25  30  35  40  45  50  55  60  65  70  75  80  85]


In [183]:
# print(df_filt_anom)
print(df_filt_anom.loc[df_filt_anom['vn_anom']==0])


              id  mm     dd    yy     lat      lon       ve      vn      spd  \
559893   7711545  11  16.75  1989   1.681  204.809 -120.571  47.363  129.540   
2653369  7704951   7   9.25  1993  37.530  234.006   24.964  10.612   27.126   
3416484  7717955   5  16.25  1994  39.524  235.040   15.204   4.717   15.919   
4006118  9421179   3  31.50  1995 -34.065   25.477   27.585  -3.208   27.771   

              hrs     corr_ve    corr_vn  wind_u10   wind_v10  filt_lat  \
559893   787842.0 -120.307778  47.286047 -3.760312   1.099331       NaN   
2653369  819774.0   24.795039  11.392249  2.413727 -11.146416       NaN   
3416484  827238.0   14.851686   4.934658  5.033059  -3.109398       NaN   
4006118  834900.0   26.944998  -3.411094  9.142892   2.901348       NaN   

         filt_lon  filt_ve  filt_vn  vn_anom  ve_anom  
559893        NaN      NaN      NaN      0.0      0.0  
2653369       NaN      NaN      NaN      0.0      0.0  
3416484       NaN      NaN      NaN      0.0      0.0  

In [185]:
# df_filt_anom.isna().any()
index = df_filt_anom['filt_lat'].index[df_filt_anom['filt_lat'].apply(np.isnan)]
print(index)

Int64Index([559893, 2653369, 3416484, 4006118], dtype='int64')


In [240]:
def compute_norm_wind_stress(df):
    df = df.copy()
    c_d_u = 0.0027 * 1/df['wind_u10'] + 0.000142 + 0.0000764 * df['wind_u10'].abs()
    c_d_v = 0.0027 * 1/df['wind_v10'] + 0.000142 + 0.0000764 * df['wind_v10'].abs()
    rho = 1.225
    tau_u10 = c_d_u * rho * df['wind_u10'] * df['wind_u10'].abs()
    tau_v10 = c_d_v * rho * df['wind_v10'] * df['wind_v10'].abs()
    df['tau_u10'] = tau_u10
    df['tau_v10'] = tau_v10
    f = 2*omega*np.sin(df['filt_lat'])
#     df['f'] = f
    df['norm_tu10'] = tau_u10/((f*tau_u10).abs())**(1/2)
    df['norm_tv10'] = tau_v10/((f*tau_v10).abs())**(1/2)
    return df

In [241]:
df_filt_anom_tau = compute_norm_wind_stress(df_filt_anom)
print(df_filt_anom_tau)

              id  mm    dd    yy     lat      lon      ve      vn     spd  \
1        7702986   3  8.25  1988  -1.242  274.772 -49.214  36.778  61.438   
2        7702986   3  8.50  1988  -1.176  274.657 -57.514  28.439  64.161   
3        7702986   3  8.75  1988  -1.131  274.548 -48.625  28.747  56.487   
4        7702986   3  9.00  1988  -1.064  274.468 -50.787  34.399  61.340   
5        7702986   3  9.25  1988  -0.997  274.351 -59.573  34.328  68.756   
...          ...  ..   ...   ...     ...      ...     ...     ...     ...   
8334588  9815710   3  8.25  1999  12.842   61.420  -6.746  12.206  13.946   
8334589  9815710   3  8.50  1999  12.872   61.388  -3.180  12.852  13.239   
8334590  9815710   3  8.75  1999  12.892   61.408   5.135  10.721  11.887   
8334591  9815710   3  9.00  1999  12.914   61.408   3.649   8.132   8.914   
8334592  9815710   3  9.25  1999  12.924   61.422   0.824   3.329   3.429   

              hrs  ...   filt_lat    filt_lon    filt_ve    filt_vn   vn_an

In [242]:
print(df_filt_anom_tau.loc[df_filt_anom_tau['norm_tu10'].apply(np.isnan)])
index = df_filt_anom_tau['norm_tu10'].index[df_filt_anom_tau['norm_tu10'].apply(np.isnan)]
print(len(index))

              id  mm     dd    yy     lat      lon       ve      vn      spd  \
559893   7711545  11  16.75  1989   1.681  204.809 -120.571  47.363  129.540   
2653369  7704951   7   9.25  1993  37.530  234.006   24.964  10.612   27.126   
3416484  7717955   5  16.25  1994  39.524  235.040   15.204   4.717   15.919   
4006118  9421179   3  31.50  1995 -34.065   25.477   27.585  -3.208   27.771   

              hrs  ...  filt_lat  filt_lon  filt_ve  filt_vn  vn_anom  \
559893   787842.0  ...       NaN       NaN      NaN      NaN      0.0   
2653369  819774.0  ...       NaN       NaN      NaN      NaN      0.0   
3416484  827238.0  ...       NaN       NaN      NaN      NaN      0.0   
4006118  834900.0  ...       NaN       NaN      NaN      NaN      0.0   

         ve_anom   tau_u10   tau_v10  norm_tu10  norm_tv10  
559893       0.0  0.005001  0.003971        NaN        NaN  
2653369      0.0  0.010313 -0.114354        NaN        NaN  
3416484      0.0  0.032986  0.005789        NaN   